## **`OPEN_DOMAIN_Q&A_SYSTEM USING WKIPEDIA`**

##### **INSTALLING NECESSARY MODULES**

In [ ]:
#install necessary packages and modules
! pip install wikipedia
! pip install transformers
! pip install sentence_transformers
! pip install spacy_universal_sentence_encoder
! python -m spacy download en_core_web_lg

##### **IMPORTING NECESSARY MODULES**

In [3]:
#importing libraries
import wikipedia
import spacy
from spacy.matcher import PhraseMatcher
import re
import spacy_universal_sentence_encoder
import pandas as pd
import numpy as np

#import transformers library
from transformers import pipeline

##### **LOAD ENGLISH LANGUAGE MODULE of SPACY**

In [4]:
#load  spacy english language module(large)
nlp = spacy.load('en_core_web_lg')

# Ssentece embedding load one of the models: ['en_use_md', 'en_use_lg', 'xx_use_md', 'xx_use_lg']
nlp1 = spacy_universal_sentence_encoder.load_model('en_use_lg')

Downloaded https://tfhub.dev/google/universal-sentence-encoder-large/5, Total size: 577.10MB



##### **GETTING USER INPUT TO RETRIEVE WIKI PAGES**

In [216]:
question = input("enter the text to search!!!")
doc1=nlp(question)
search_text=''
for token in doc1:
  if token.pos_ in ('PROPN', 'NUM', 'VERB', 'NOUN', 'ADJ'):
    search_text +=' '+ token.text

search_text=search_text.strip()

enter the text to search!!!who is steven spielberg?


In [254]:
import time
start_time = time.time()
#check process running time

In [255]:
doc1.text

'who is steven spielberg?'

In [256]:
for token in doc1:
  print(token,token.pos_)

who PRON
is AUX
steven PROPN
spielberg PROPN
? PUNCT


In [257]:
search_text

'steven spielberg'

##### **Wiki PAGES RETRIEVING ( DATA IMPORTING )**

In [258]:
#search and retrieve wiki pages using pytohn wikipedia API module
#pass the neceesary pages to retrieve (example:1 or 2 or 3)
search_results = wikipedia.search(search_text,results = 1)

search_results_pages=[]

# add pages to list
for docs in search_results:
  search_results_pages.append(wikipedia.page(docs, auto_suggest=False))

In [259]:
#lisitng search pages
search_results_pages

[<WikipediaPage 'Steven Spielberg'>]

##### **ADDING MATCHER AND PERFORM REQUIRED CONTEXT EXTRACTION( DATA SELECTION )**

In [260]:
#extracting sentence with the required matches with spacy matcher
phrase_matcher = PhraseMatcher(nlp.vocab)
phrases = [x for x in search_text.split()]
patterns = [nlp(text) for text in phrases]
Upper_Case = [nlp(text.upper()) for text in phrases]
Capitalize = [nlp(text.capitalize()) for text in phrases]
Lower = [nlp(text.lower()) for text in phrases]
phrase_matcher.add('Matches', None, *patterns)
phrase_matcher.add('Upper_Case', None, *Upper_Case)
phrase_matcher.add('Capitalize', None, *Capitalize)
phrase_matcher.add('Lower', None, *Lower)
final_docs=''

In [261]:
# creating a pandas dataframe to store matches on wikipedia articles.
df = pd.DataFrame({'Wiki_Page': pd.Series(dtype='str'),
                   'Wikipedia_Setences': pd.Series(dtype='str')})
df_columns=list(df)
df_columns

['Wiki_Page', 'Wikipedia_Setences']

In [262]:
#function to find and extract paragraph of doc
def paragraphs(document):
    start = 0
    for token in document:
        if token.is_space and token.text.count("\n") > 1:
            yield document[start:token.i]
            start = token.i
    yield document[start:]

In [263]:
#retrieve matching context
for x in search_results_pages:
  final_docs=''
  for y in paragraphs(nlp(x.content)):
    b=nlp(y.text)
    for match_id, start, end in phrase_matcher(b):
      if nlp.vocab.strings[match_id] in ["Matches","Upper_Case","Capitalize","Lower"]:
        final_docs=b.text
        doc2=nlp(final_docs)
        values=[x,final_docs]
        zipped = zip(df_columns, values)
        append_dictionary = dict(zipped)
        df = df.append(append_dictionary, ignore_index=True)

In [264]:
#drop duplicates
df = df.loc[df.astype(str).drop_duplicates().index]
df

,Wiki_Page,Wikipedia_Setences
0,<WikipediaPage 'Steven Spielberg'>,"Steven Allan Spielberg (; born December 18, 19..."
8,<WikipediaPage 'Steven Spielberg'>,\n\n\n== Early life and background ==\nSteven ...
25,<WikipediaPage 'Steven Spielberg'>,\n\n\n=== 1969–1974: Entering film and televis...
28,<WikipediaPage 'Steven Spielberg'>,"\n\nWhen I began to work with Steven, I unders..."
40,<WikipediaPage 'Steven Spielberg'>,\n\n\n=== 1975–1983: Film breakthrough ===\nPr...
58,<WikipediaPage 'Steven Spielberg'>,\n\n\n=== 1984–1990: From producing to directi...
81,<WikipediaPage 'Steven Spielberg'>,\n\nAfter a brief setback in which the directo...
102,<WikipediaPage 'Steven Spielberg'>,\n\n\n=== 1999–2007: Further directorial work ...
122,<WikipediaPage 'Steven Spielberg'>,\n\n\n=== 2008–2015: Return to film ===\nSpiel...
133,<WikipediaPage 'Steven Spielberg'>,"\n\n\n=== 2016–present ===\nIn 2016, Spielberg..."


In [265]:
#write to CSV
df.to_csv("Matching Wiki context.csv")

In [266]:
removed_sp_char_Df = df
removed_sp_char_Df['Wikipedia_Setences'] = [re.sub(r"[^a-zA-Z0-9]+", ' ', doc) for doc in df['Wikipedia_Setences']]

In [267]:
removed_sp_char_Df.reset_index(inplace=True)
removed_sp_char_Df.drop(columns=['index'],inplace=True)
removed_sp_char_Df

,Wiki_Page,Wikipedia_Setences
0,<WikipediaPage 'Steven Spielberg'>,Steven Allan Spielberg born December 18 1946 i...
1,<WikipediaPage 'Steven Spielberg'>,Early life and background Steven Allan Spielb...
2,<WikipediaPage 'Steven Spielberg'>,1969 1974 Entering film and television Spielb...
3,<WikipediaPage 'Steven Spielberg'>,When I began to work with Steven I understood...
4,<WikipediaPage 'Steven Spielberg'>,1975 1983 Film breakthrough Producers Richard...
5,<WikipediaPage 'Steven Spielberg'>,1984 1990 From producing to directing In 1984...
6,<WikipediaPage 'Steven Spielberg'>,After a brief setback in which the director f...
7,<WikipediaPage 'Steven Spielberg'>,1999 2007 Further directorial work In 2001 Sp...
8,<WikipediaPage 'Steven Spielberg'>,2008 2015 Return to film Spielberg returned t...
9,<WikipediaPage 'Steven Spielberg'>,2016 present In 2016 Spielberg made The BFG a...


In [268]:
#write to CSV for reference
removed_sp_char_Df.to_csv('Cleaned_context.csv')

In [269]:
# creating a pandas dataframe to store Matches along with similarity score based on sentence embedding techniques
Context_Similarity = pd.DataFrame({'Context': pd.Series(dtype='str'),
                   'Similarity': pd.Series(dtype='str')})

columns=list(Context_Similarity)
columns

['Context', 'Similarity']

##### **PERFORM DATA WRANGLING/FEATURE EXTRACTION - STOP WORD REMOVAL, LEMMATIZATION, SENTENCE EMEDING & NECESSARY CONTEXT EXTRACTION/RANKING**

In [270]:
#get the stop words to remove
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

In [271]:
#remove punctutations and other unnecessary stuff from doc
#corpus = [re.sub(r"[^a-zA-Z0-9]+", ' ', doc) for doc in df['Wikipedia_Setences']]
corpus=removed_sp_char_Df['Wikipedia_Setences']
corpus

0     Steven Allan Spielberg born December 18 1946 i...
1      Early life and background Steven Allan Spielb...
2      1969 1974 Entering film and television Spielb...
3      When I began to work with Steven I understood...
4      1975 1983 Film breakthrough Producers Richard...
5      1984 1990 From producing to directing In 1984...
6      After a brief setback in which the director f...
7      1999 2007 Further directorial work In 2001 Sp...
8      2008 2015 Return to film Spielberg returned t...
9      2016 present In 2016 Spielberg made The BFG a...
10     Prospective projects In May 2009 Spielberg bo...
11     Other ventures The director has been an avid ...
12     Influences Spielberg has cited Frank Capra s ...
13     Method and themes Spielberg often uses storyb...
14     Collaborators Janusz Kami ski has served as a...
15     John Williams The Sugarland Express began Spi...
16     Personal life Spielberg met actress Amy Irvin...
17     Political views Spielberg has usually sup

In [272]:
#do lemmatization inorder to convert to canonical form & remove stop words.
lemma_process=lambda text: " ".join([token.lemma_ for token in nlp(text) if not token.is_stop])

# list created post lemmatization & stop word removal
canonical=[]

#loop over corpus and perform lemmatization & stop word removal
for x in corpus:
  canonical.append(lemma_process(x))

#list in canonical form
canonical

['Steven Allan Spielberg bear December 18 1946 american film director producer screenwriter begin career New Hollywood era currently commercially successful director Spielberg recipient accolade include Academy Awards Best Director Kennedy Center honor Cecil B DeMille Award AFI Life Achievement Award Spielberg bear Cincinnati Ohio grow Phoenix Arizona later move California study film college direct television episode minor film Universal Studios household direct 1975 s summer blockbuster Jaws direct box office success close encounter Kind 1977 e T Extra terrestrial 1982 adventure film Indiana Jones series Spielberg later explore drama Color Purple 1985 Empire Sun 1987 brief hiatus direct box office success acclaimed science fiction action film Jurassic Park holocaust drama Schindler s List 1993 1998 direct World War II epic save Private Ryan critical commercial success Spielberg continue 2000s science fiction include Artificial Intelligence 2001 Minority Report 2002 War Worlds 2005 dir

In [273]:
#add doc1 to nlp1 object
doc_1 = nlp1(question)

for x in corpus:
  doc_2 = nlp1(lemma_process(x))
  
  #compute sentence semantic similarity between quesiton and contexts using Universal Sentence Encoder
  similarity_rate=doc_1.similarity(doc_2)
  values=[x,similarity_rate, ]
  zipped = zip(Context_Similarity_columns, values)
  append_dictionary = dict(zipped)
  #append values to dataframe
  Context_Similarity = Context_Similarity.append(append_dictionary, ignore_index=True)

Context_Similarity.sort_values(by=['Similarity'], ascending=False, inplace=True)

#filter context if it is of very less length, since it wont be of no use for our model to get context out of it.
length_of_question = len(question)
Context_Similarity = Context_Similarity[Context_Similarity['Context'].apply(len) > 2*length_of_question]

#retreive only top10 context from the dataframe
top10=Context_Similarity.iloc[:10,:]

In [274]:
#merge to retrive wiki page info
top_10 = top10.merge(removed_sp_char_Df,left_index=True, right_index=True)

In [275]:
top_10.drop(columns=['Wikipedia_Setences'],inplace=True)
top_10.reset_index(inplace=True)
top_10.drop(columns=['index'],inplace=True)
top_10

,Context,Similarity,Wiki_Page
0,Steven Spielberg at IMDb Steven Spielberg at ...,0.670487,<WikipediaPage 'Steven Spielberg'>
1,See also Directors with two films rated A by ...,0.652816,<WikipediaPage 'Steven Spielberg'>
2,Steven Allan Spielberg born December 18 1946 i...,0.632401,<WikipediaPage 'Steven Spielberg'>
3,Collaborators Janusz Kami ski has served as a...,0.541659,<WikipediaPage 'Steven Spielberg'>
4,Influences Spielberg has cited Frank Capra s ...,0.524965,<WikipediaPage 'Steven Spielberg'>
5,Legacy A notable figure of the New Hollywood ...,0.516514,<WikipediaPage 'Steven Spielberg'>
6,Method and themes Spielberg often uses storyb...,0.514598,<WikipediaPage 'Steven Spielberg'>
7,Spielberg has won three Academy Awards He rec...,0.507421,<WikipediaPage 'Steven Spielberg'>
8,Prolific in film since the 1960s Spielberg ha...,0.502446,<WikipediaPage 'Steven Spielberg'>
9,Political views Spielberg has usually support...,0.487830,<WikipediaPage 'Steven Spielberg'>


In [276]:
#wriite to csv
top_10.to_csv("TOP_10_Similarities.csv")

##### **TRYING DIFFERENT ML MODEL FROM TRANSFORMERS LIBRARY**

###### **ROBERTA**

In [277]:
#trying different exisiting pre trained models in hugging face
roberta_findings=[]
#Model 1
qa_pipeline = pipeline("question-answering",model="deepset/roberta-base-squad2",tokenizer="deepset/roberta-base-squad2", revision="v1.0")

#looping through all context
for x,context in enumerate(top_10['Context']):
  prediction = qa_pipeline({'context': context,'question': question})
  roberta_findings.append([prediction['answer'],prediction['score'],top_10.iloc[x,2],context])

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [278]:
##extract only the top 3 results from the model
from operator import itemgetter
roberta_final_results = sorted(roberta_findings, key=itemgetter(1), reverse=True)[:3]

In [279]:
#convert to dataframe and save to csv as top3_predictions
Roberta_Final_predictions=pd.DataFrame(roberta_final_results,columns=['Prediction','Score','Wiki_Page','Context'])
Roberta_Final_predictions

,Prediction,Score,Wiki_Page,Context
0,the greatest filmmaker in history,0.834168,<WikipediaPage 'Steven Spielberg'>,Legacy A notable figure of the New Hollywood ...
1,an American film director producer and screenw...,0.616803,<WikipediaPage 'Steven Spielberg'>,Steven Allan Spielberg born December 18 1946 i...
2,The director,0.481511,<WikipediaPage 'Steven Spielberg'>,Method and themes Spielberg often uses storyb...


In [280]:
#save to csv
Roberta_Final_predictions.to_csv('Roberta_Final_predictions.csv')

In [281]:
end_time = time.time()
print("---program execution time %s seconds ---" % (time.time() - start_time))

---program execution time 77.74932169914246 seconds ---


###### **BERT-BASE-CASED**

In [282]:
#Model 2
bert_findings=[]

qa_pipeline = pipeline("question-answering",model="deepset/bert-base-cased-squad2",tokenizer="deepset/bert-base-cased-squad2")

#looping through all context
for x,context in enumerate(top_10['Context']):
  prediction = qa_pipeline({'context': context,'question': question})
  bert_findings.append([prediction['answer'],prediction['score'],top_10.iloc[x,2],context])

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [283]:
##extract only the top 3 results from the model
from operator import itemgetter
bert_final_results = sorted(bert_findings, key=itemgetter(1), reverse=True)[:3]

In [284]:
Bert_Final_predictions= pd.DataFrame(bert_final_results,columns=['Prediction','Score','Wiki_Page','Context'])
Bert_Final_predictions

,Prediction,Score,Wiki_Page,Context
0,former President Bill Clinton,0.537619,<WikipediaPage 'Steven Spielberg'>,Political views Spielberg has usually support...
1,former U S President Bill Clinton,0.497652,<WikipediaPage 'Steven Spielberg'>,Spielberg has won three Academy Awards He rec...
2,his father a science fiction fan,0.488865,<WikipediaPage 'Steven Spielberg'>,Method and themes Spielberg often uses storyb...


In [285]:
#save to csv
Bert_Final_predictions.to_csv('Bert_Final_predictions.csv')

###### **NOTE:** *FOR MOST CASES TOP RESULT WERE FINE, FOR SOME CASES ANSWERS WERE FOUND IN OTHER RESULT(2nd or 3rd).*

### **Passing the whole contents of a Wikipedia page as context to Roberta Model and testing the accuracy of predictions**

In [286]:
#type the question to search
question = input("enter the text to search!!!")
doc1=nlp(question)
search_text=''
for token in doc1:
  if token.pos_ in ('PROPN', 'NUM', 'VERB', 'NOUN', 'ADJ'):
    search_text +=' '+ token.text

search_text=search_text.strip()

enter the text to search!!!who is steven spielberg?


In [287]:
import time
start_time = time.time()
#check process running time

In [288]:
doc1.text

'who is steven spielberg?'

In [289]:
for token in doc1:
  print(token,token.pos_)

who PRON
is AUX
steven PROPN
spielberg PROPN
? PUNCT


In [290]:
search_text

'steven spielberg'

In [291]:
#search and retrieve wiki pages using pytohn wikipedia API module
#pass the neceesary pages to retrieve (example:1 or 2 or 3)
search_results = wikipedia.search(search_text,results = 1)

search_results_pages=[]

# add pages to list
for docs in search_results:
  search_results_pages.append(wikipedia.page(docs, auto_suggest=False))

In [292]:
#wiki page
search_results_pages[0]

<WikipediaPage 'Steven Spielberg'>

In [293]:
#passing entire page to Roberta Model and check out predictions
prediction = qa_pipeline({'context': search_results_pages[0].content,'question': question})
prediction

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


{'answer': 'Oskar Schindler',
 'end': 22286,
 'score': 0.5922462940216064,
 'start': 22271}

In [294]:
end_time = time.time()
print("---program execution time %s seconds ---" % (time.time() - start_time))

---program execution time 87.84971785545349 seconds ---


**The Results were appropiate for few cases and were totally irrelevant for many other.**

*Also, this process was slower(87 seconds) compared to detailed steps done along with IO operations to disk(77 seconds).*